In [6]:
import os

os.makedirs("/Users/tongaimutengwa/Projects/homework5", exist_ok=True)
home = "/Users/tongaimutengwa/Projects/homework5"
os.chdir(home)
!pwd

/Users/tongaimutengwa/Projects/homework5


In [7]:
#!curl -O https://files.pythonhosted.org/packages/py3/s/scikit_learn/scikit_learn-1.6.1-py3-none-any.whl

In [8]:
!ls

scikit_learn-1.6.1-py3-none-any.whl


In [14]:
import hashlib

# Define the file paths
# Filter out all paths that are directories
actual_files = [path for path in file_path if os.path.isfile(path)]

# Compute and print the SHA256 hash for each file
for path in actual_files:
    with open(path, "rb") as f:
        file_bytes = f.read()
        hash_value = hashlib.sha256(file_bytes).hexdigest()
        print(f"✅ SHA256 hash for {file.split('/')[-1]}:")
        print("sha256:" + hash_value + "\n")

✅ SHA256 hash for .venv:
sha256:e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855



In [ ]:
#!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-10-30 10:12:31--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 4.237.22.38
Connecting to github.com (github.com)|4.237.22.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-30 10:12:32--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin.2’


In [26]:
ls -la

total 8
drwxr-xr-x@  5 tongaimutengwa  staff   160 Oct 30 10:14 ./
drwxr-xr-x@ 18 tongaimutengwa  staff   576 Oct 30 09:04 ../
drwxr-xr-x@  8 tongaimutengwa  staff   256 Oct 30 09:30 .venv/
-rw-r--r--@  1 tongaimutengwa  staff  1300 Oct 30 10:14 pipeline_v1.bin
-rw-r--r--@  1 tongaimutengwa  staff     0 Oct 30 09:33 scikit_learn-1.6.1-py3-none-any.whl


In [27]:
import pickle

with open("pipeline_v1.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

In [28]:
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

X = dv.transform([record])

y_pred = model.predict_proba(X)[:, 1]

print(f"Prediction probability: {y_pred[0]:.4f}")

Prediction probability: 0.5336


In [30]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle

# Load the model and vectorizer
with open("pipeline_v1.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

# Define the input data model
class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

# Create the FastAPI app
app = FastAPI()

@app.post("/predict")
def predict(client: Client):
    record = client.model_dump()
    X = dv.transform([record])
    y_pred = model.predict_proba(X)[:, 1]
    return {"prediction_probability": y_pred[0]}

# Run fastapi server Subtask: Start the FastAPI application using uvicorn in a way that it runs in the background or in a separate thread

In [31]:
import threading
import time
import uvicorn

def run_server():
    """Runs the uvicorn server."""
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Create and start the server thread
server_thread = threading.Thread(target=run_server)
server_thread.start()

# Allow the server to initialize
time.sleep(1)

print("FastAPI server is running in a separate thread.")

INFO:     Started server process [45469]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


FastAPI server is running in a separate thread.


INFO:     127.0.0.1:63388 - "POST /predict HTTP/1.1" 200 OK


In [32]:
import requests

url = "http://localhost:8000/predict"  # FastAPI app is running on localhost:8000
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())

{'prediction_probability': 0.5340417283801275}


In [33]:
!docker images agrigorev/zoomcamp-model:2025

REPOSITORY                 TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-model   2025      14d79fde0bbf   8 days ago   181MB
